文中のすべての名詞を含む文節に対し，その文節から構文木の根に至るパスを抽出せよ． ただし，構文木上のパスは以下の仕様を満たすものとする．

- 各文節は（表層形の）形態素列で表現する
- パスの開始文節から終了文節に至るまで，各文節の表現を"->"で連結する

「吾輩はここで始めて人間というものを見た」という文（neko.txt.cabochaの8文目）から，次のような出力が得られるはずである．

```
吾輩は -> 見た
ここで -> 始めて -> 人間という -> ものを -> 見た
人間という -> ものを -> 見た
ものを -> 見た
```

In [1]:
import re

In [2]:
# 区切り文字
separator = re.compile('\t|,')

# 係り受け
dependancy = re.compile(r'''(?:\*\s\d+\s) # キャプチャ対象外
                            (-?\d+)       # 数字(係り先)
                          ''', re.VERBOSE)

In [3]:
class Morph:
    def __init__(self, line):
        
        #タブとカンマで分割
        cols = separator.split(line)
        
        self.surface = cols[0] # 表層形(surface)
        self.base = cols[7]    # 基本形(base)
        self.pos = cols[1]     # 品詞(pos)
        self.pos1 = cols[2]    # 品詞細分類1(pos1)

In [4]:
class Chunk:
    def __init__(self, morphs, dst):
        self.morphs = morphs
        self.dst  = dst  # 係り先文節インデックス番号
        
        self.phrase = ''
        self.noun = False
        
        for morph in morphs:
            if morph.pos != '記号':
                self.phrase += morph.surface # 記号以外の場合文節作成
            if morph.pos == '名詞':
                self.noun = True

In [5]:
%time

morphs = []
chunks = []
sentences = []

with open('./neko.txt.cabocha') as f:
    
    for line in f:
        dependancies = dependancy.match(line)
        
        # EOSまたは係り受け解析結果でない場合
        if not (line == 'EOS\n' or dependancies):
            morphs.append(Morph(line))
            
        # EOSまたは係り受け解析結果で、形態素解析結果がある場合
        elif len(morphs) > 0:
            chunks.append(Chunk(morphs, dst))
            morphs = []
       
        # 係り受け結果の場合
        if dependancies:
            dst = int(dependancies.group(1))
        
        # EOSで係り受け結果がある場合
        if line == 'EOS\n' and len(chunks) > 0:
            sentences.append(chunks)
            chunks = []

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 10 µs


In [6]:
for i, sentence in enumerate(sentences):
    for chunk in sentence:
        if chunk.noun and chunk.dst != -1:
            line = chunk.phrase
            current_chunk = chunk
            while current_chunk.dst != -1:
                line = line + ' -> ' + sentence[current_chunk.dst].phrase
                current_chunk = sentence[current_chunk.dst]
            print(i, '\t',line)
    # 多いので制限
    if i > 10:
        break

2 	 名前は -> 無い
3 	 どこで -> 生れた -> かとんと -> つかぬ
3 	 かとんと -> つかぬ
3 	 見当が -> つかぬ
4 	 何でも -> 薄暗い -> 泣いて -> 記憶している
4 	 した所で -> 泣いて -> 記憶している
4 	 いた事だけは -> 記憶している
5 	 吾輩は -> 見た
5 	 ここで -> 始めて -> 人間という -> ものを -> 見た
5 	 人間という -> ものを -> 見た
5 	 ものを -> 見た
6 	 あとで -> 聞くと -> そうだ
6 	 それは -> そうだ
6 	 書生という -> 人間中で -> 種族であった -> そうだ
6 	 人間中で -> 種族であった -> そうだ
6 	 一番 -> 獰悪な -> 種族であった -> そうだ
6 	 獰悪な -> 種族であった -> そうだ
6 	 種族であった -> そうだ
7 	 書生というのは -> 話である
7 	 我々を -> 捕えて -> 煮て -> 食うという -> 話である
8 	 当時は -> なかったから -> 思わなかった
8 	 何という -> 考も -> なかったから -> 思わなかった
8 	 考も -> なかったから -> 思わなかった
9 	 彼の -> 掌に -> 載せられて -> 持ち上げられた -> 時 -> フワフワした -> 感じが -> あったばかりである
9 	 掌に -> 載せられて -> 持ち上げられた -> 時 -> フワフワした -> 感じが -> あったばかりである
9 	 スーと -> 持ち上げられた -> 時 -> フワフワした -> 感じが -> あったばかりである
9 	 時 -> フワフワした -> 感じが -> あったばかりである
9 	 感じが -> あったばかりである
10 	 掌の -> 上で -> 落ちついて -> 見たのが -> 人間という -> ものの -> 見始であろう
10 	 上で -> 落ちついて -> 見たのが -> 人間という -> ものの -> 見始であろう
10 	 書生の -> 顔を -> 見たのが -> 人間という -> ものの -> 見始であろう
10 	 顔を -> 見たのが -> 人間という -> ものの -> 見始であろ